# Ⅰ.유지이탈 분류 전처리

## 가. MYSQL에서 불러오기

## 1. 채널 데이터 불러오기

In [1]:
import pandas as pd
import pymysql
import numpy as np


conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select cust, chnl_dv, count(chnl_dv) from 엘페이이용_view group by cust, chnl_dv order by cust;")
result = cur.fetchall()
conn.commit()
conn.close()
df_ch = pd.DataFrame(result)

In [2]:
df_ch.columns=['cust','채널','사용량']
df_ch

,cust,채널,사용량
0,M000136117,1,1
1,M000136117,2,3
2,M000419293,1,16
3,M000419293,2,26
4,M000494848,1,6
...,...,...,...
13046,M999599111,2,77
13047,M999770689,1,74
13048,M999770689,2,5
13049,M999962961,1,9


## 2. 제휴사 데이터 불러오기

In [3]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select cust, cop_c, count(cop_c) from 엘페이이용_view group by cust, cop_c order by cust;")
result = cur.fetchall()
conn.commit()
conn.close()
df_cop = pd.DataFrame(result)

In [4]:
df_cop.columns=['cust','제휴사','이용량']
df_cop

,cust,제휴사,이용량
0,M000136117,A01,1
1,M000136117,A06,2
2,M000136117,D01,1
3,M000419293,A01,15
4,M000419293,A02,1
...,...,...,...
30263,M999962961,A02,1
30264,M999962961,A03,1
30265,M999962961,A06,6
30266,M999962961,C02,1


## 3. 총사용량 데이터 불러오기

In [5]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select cust, count(*) from 엘페이이용_view group by cust order by cust;")
result = cur.fetchall()
conn.commit()
conn.close()
df_total = pd.DataFrame(result)

In [6]:
df_total.columns=['cust','총사용량']

df_total

,cust,총사용량
0,M000136117,4
1,M000419293,42
2,M000494848,94
3,M000557840,38
4,M000871427,5
...,...,...
8901,M999227380,2
8902,M999268192,30
8903,M999599111,77
8904,M999770689,79


## 4. 성별 데이터 불러오기

In [7]:
import pandas as pd
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select a.cust, ma_fem_dv from 엘페이이용_view a join 고객정보 b on a.cust = b.cust group by cust order by cust;")
result = cur.fetchall()
conn.commit()
conn.close()
df_sex = pd.DataFrame(result)

In [8]:
df_sex.columns=['cust','ma_fem_dv']
for i in range(len(df_sex)):
    if df_sex["ma_fem_dv"].iloc[i] == '여성':
        df_sex["ma_fem_dv"].iloc[i] = 1
    else:
        df_sex["ma_fem_dv"].iloc[i] = 0

In [9]:
df_sex

,cust,ma_fem_dv
0,M000136117,1
1,M000419293,1
2,M000494848,1
3,M000557840,0
4,M000871427,1
...,...,...
8901,M999227380,1
8902,M999268192,1
8903,M999599111,1
8904,M999770689,1


## 5. count_days 데이터 불러오기

In [10]:
import pandas as pd
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select cust, count_days from 엘페이이용정보 order by cust;")
result = cur.fetchall()
conn.commit()
conn.close()
df_count_days = pd.DataFrame(result)

In [11]:
df_count_days.columns=['cust','count_days']

In [12]:
df_count_days

,cust,count_days
0,M000136117,4
1,M000419293,30
2,M000494848,75
3,M000557840,37
4,M000871427,3
...,...,...
8901,M999227380,2
8902,M999268192,23
8903,M999599111,67
8904,M999770689,67


## 나. 모델에 넣을 DataFrame 만들기

## 1. pivot table 만들기
### - 채널
### - 제휴사

In [13]:
df_ch_pivot=df_ch.pivot('cust','채널','사용량')

for i in range(len(df_ch_pivot)):
    for j in range(len(df_ch_pivot.columns)):
        df_ch_pivot.iloc[i][j]=round(df_ch_pivot.iloc[i][j]/df_total.iloc[i][1],2)
df_ch_pivot=df_ch_pivot.fillna(0)

df_ch_pivot.columns=['offline','online']

In [14]:
df_list = df_cop['제휴사'].values.tolist()
column_name = []
for v in df_list:
    if v not in column_name:
        column_name.append(v)
column_name.sort()

df_cop_pivot=df_cop.pivot('cust','제휴사','이용량')
df_cop_pivot

for i in range(len(df_cop_pivot)):
    for j in range(len(df_cop_pivot.columns)):
        df_cop_pivot.iloc[i][j]=round(df_cop_pivot.iloc[i][j]/df_total.iloc[i][1],2)
df_cop_pivot=df_cop_pivot.fillna(0)
df_cop_pivot.columns=column_name
df_cop_pivot

,A01,A02,A03,A04,A05,A06,B01,C01,C02,D01,D02,E01,L00,L01
cust,,,,,,,,,,,,,,
M000136117,0.25,0.00,0.00,0.0,0.00,0.50,0.0,0.00,0.00,0.25,0.00,0.0,0.00,0.00
M000419293,0.36,0.02,0.00,0.0,0.00,0.00,0.0,0.05,0.00,0.10,0.00,0.0,0.02,0.45
M000494848,0.05,0.00,0.00,0.0,0.00,0.43,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.51
M000557840,0.00,1.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00
M000871427,0.20,0.00,0.00,0.0,0.00,0.00,0.0,0.60,0.00,0.00,0.20,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999227380,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.0,0.00,0.00
M999268192,0.00,0.00,0.00,0.0,0.03,0.03,0.0,0.00,0.00,0.03,0.00,0.0,0.37,0.53
M999599111,0.00,0.00,0.00,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.97


## 2. cust를 index로

In [15]:
df_total=df_total.set_index('cust')
df_sex=df_sex.set_index('cust')
df_count_days=df_count_days.set_index('cust')

## 3. dataframe 합치기 및 df_final 정의
### - dataframe label column 생성

In [16]:
df_final=pd.concat([df_sex,df_cop_pivot,df_ch_pivot,df_count_days],axis=1)

In [17]:
df_final["label"]=np.NaN

In [18]:
df_final

,ma_fem_dv,A01,A02,A03,A04,A05,A06,B01,C01,C02,D01,D02,E01,L00,L01,offline,online,count_days,label
cust,,,,,,,,,,,,,,,,,,,
M000136117,1,0.25,0.00,0.00,0.0,0.00,0.50,0.0,0.00,0.00,0.25,0.00,0.0,0.00,0.00,0.25,0.75,4,NaN
M000419293,1,0.36,0.02,0.00,0.0,0.00,0.00,0.0,0.05,0.00,0.10,0.00,0.0,0.02,0.45,0.38,0.62,30,NaN
M000494848,1,0.05,0.00,0.00,0.0,0.00,0.43,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.51,0.06,0.94,75,NaN
M000557840,0,0.00,1.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,1.00,0.00,37,NaN
M000871427,1,0.20,0.00,0.00,0.0,0.00,0.00,0.0,0.60,0.00,0.00,0.20,0.0,0.00,0.00,0.80,0.20,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999227380,1,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.0,0.00,0.00,0.00,1.00,2,NaN
M999268192,1,0.00,0.00,0.00,0.0,0.03,0.03,0.0,0.00,0.00,0.03,0.00,0.0,0.37,0.53,0.03,0.97,23,NaN
M999599111,1,0.00,0.00,0.00,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.97,0.00,1.00,67,NaN


## 4. 재이용기간이 45일 이하 고객 추출후 label 값에 이탈:0, 유지:1 넣기


In [19]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select a.cust from 엘페이이용_view a inner join 엘페이이용정보 b on a.cust = b.cust where max_days <= 45 group by cust order by cust, cop_c;")
result = cur.fetchall()
conn.commit()
conn.close()
df_stay_45 = pd.DataFrame(result)
df_stay_45.columns=['고객번호']
df_stay_45
df_45 = df_final

In [20]:
for i in range(len(df_stay_45)):
    df_45.loc[[df_stay_45.iloc[i][0]],['label']] = 1
df_45=df_45.fillna(0)

In [21]:
df_final

,ma_fem_dv,A01,A02,A03,A04,A05,A06,B01,C01,C02,D01,D02,E01,L00,L01,offline,online,count_days,label
cust,,,,,,,,,,,,,,,,,,,
M000136117,1,0.25,0.00,0.00,0.0,0.00,0.50,0.0,0.00,0.00,0.25,0.00,0.0,0.00,0.00,0.25,0.75,4,NaN
M000419293,1,0.36,0.02,0.00,0.0,0.00,0.00,0.0,0.05,0.00,0.10,0.00,0.0,0.02,0.45,0.38,0.62,30,1.0
M000494848,1,0.05,0.00,0.00,0.0,0.00,0.43,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.51,0.06,0.94,75,1.0
M000557840,0,0.00,1.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,1.00,0.00,37,NaN
M000871427,1,0.20,0.00,0.00,0.0,0.00,0.00,0.0,0.60,0.00,0.00,0.20,0.0,0.00,0.00,0.80,0.20,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999227380,1,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.0,0.00,0.00,0.00,1.00,2,1.0
M999268192,1,0.00,0.00,0.00,0.0,0.03,0.03,0.0,0.00,0.00,0.03,0.00,0.0,0.37,0.53,0.03,0.97,23,NaN
M999599111,1,0.00,0.00,0.00,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.97,0.00,1.00,67,1.0


In [22]:
df_45=df_45.fillna(0)

In [23]:
df_final

,ma_fem_dv,A01,A02,A03,A04,A05,A06,B01,C01,C02,D01,D02,E01,L00,L01,offline,online,count_days,label
cust,,,,,,,,,,,,,,,,,,,
M000136117,1,0.25,0.00,0.00,0.0,0.00,0.50,0.0,0.00,0.00,0.25,0.00,0.0,0.00,0.00,0.25,0.75,4,NaN
M000419293,1,0.36,0.02,0.00,0.0,0.00,0.00,0.0,0.05,0.00,0.10,0.00,0.0,0.02,0.45,0.38,0.62,30,1.0
M000494848,1,0.05,0.00,0.00,0.0,0.00,0.43,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.51,0.06,0.94,75,1.0
M000557840,0,0.00,1.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,1.00,0.00,37,NaN
M000871427,1,0.20,0.00,0.00,0.0,0.00,0.00,0.0,0.60,0.00,0.00,0.20,0.0,0.00,0.00,0.80,0.20,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999227380,1,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.0,0.00,0.00,0.00,1.00,2,1.0
M999268192,1,0.00,0.00,0.00,0.0,0.03,0.03,0.0,0.00,0.00,0.03,0.00,0.0,0.37,0.53,0.03,0.97,23,NaN
M999599111,1,0.00,0.00,0.00,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.97,0.00,1.00,67,1.0


## 5. df_final 초기화

In [24]:
df_final=pd.concat([df_sex,df_cop_pivot,df_ch_pivot,df_count_days],axis=1)
df_final["label"]=np.NaN

## 6. 재이용기간이 50일 이하 고객 추출후 label 값에 이탈:0, 유지:1 넣기


In [25]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='lpoint', db='lpoint', charset='utf8')
cur = conn.cursor()
cur.execute("select a.cust from 엘페이이용_view a inner join 엘페이이용정보 b on a.cust = b.cust where max_days <= 50 group by cust order by cust, cop_c;")
result = cur.fetchall()
conn.commit()
conn.close()
df_stay_50 = pd.DataFrame(result)
df_stay_50.columns=['고객번호']
df_50 = df_final

In [26]:
df_50

,ma_fem_dv,A01,A02,A03,A04,A05,A06,B01,C01,C02,D01,D02,E01,L00,L01,offline,online,count_days,label
cust,,,,,,,,,,,,,,,,,,,
M000136117,1,0.25,0.00,0.00,0.0,0.00,0.50,0.0,0.00,0.00,0.25,0.00,0.0,0.00,0.00,0.25,0.75,4,NaN
M000419293,1,0.36,0.02,0.00,0.0,0.00,0.00,0.0,0.05,0.00,0.10,0.00,0.0,0.02,0.45,0.38,0.62,30,NaN
M000494848,1,0.05,0.00,0.00,0.0,0.00,0.43,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.51,0.06,0.94,75,NaN
M000557840,0,0.00,1.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,1.00,0.00,37,NaN
M000871427,1,0.20,0.00,0.00,0.0,0.00,0.00,0.0,0.60,0.00,0.00,0.20,0.0,0.00,0.00,0.80,0.20,3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999227380,1,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.0,0.00,0.00,0.00,1.00,2,NaN
M999268192,1,0.00,0.00,0.00,0.0,0.03,0.03,0.0,0.00,0.00,0.03,0.00,0.0,0.37,0.53,0.03,0.97,23,NaN
M999599111,1,0.00,0.00,0.00,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.97,0.00,1.00,67,NaN


In [27]:
df_stay_50

,고객번호
0,M000419293
1,M000494848
2,M000557840
3,M000986912
4,M001444118
...,...
4732,M998980052
4733,M999002436
4734,M999105944
4735,M999227380


In [28]:
for i in range(len(df_stay_50)):
    df_50.loc[[df_stay_50.iloc[i][0]],['label']] = 1
    
df_50=df_50.fillna(0)


In [29]:
df_50

,ma_fem_dv,A01,A02,A03,A04,A05,A06,B01,C01,C02,D01,D02,E01,L00,L01,offline,online,count_days,label
cust,,,,,,,,,,,,,,,,,,,
M000136117,1,0.25,0.00,0.00,0.0,0.00,0.50,0.0,0.00,0.00,0.25,0.00,0.0,0.00,0.00,0.25,0.75,4,0.0
M000419293,1,0.36,0.02,0.00,0.0,0.00,0.00,0.0,0.05,0.00,0.10,0.00,0.0,0.02,0.45,0.38,0.62,30,1.0
M000494848,1,0.05,0.00,0.00,0.0,0.00,0.43,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.51,0.06,0.94,75,1.0
M000557840,0,0.00,1.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,1.00,0.00,37,1.0
M000871427,1,0.20,0.00,0.00,0.0,0.00,0.00,0.0,0.60,0.00,0.00,0.20,0.0,0.00,0.00,0.80,0.20,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M999227380,1,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.0,0.00,0.00,0.00,1.00,2,1.0
M999268192,1,0.00,0.00,0.00,0.0,0.03,0.03,0.0,0.00,0.00,0.03,0.00,0.0,0.37,0.53,0.03,0.97,23,0.0
M999599111,1,0.00,0.00,0.00,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.0,0.01,0.97,0.00,1.00,67,1.0


# Ⅱ. 이탈유지분류_로지스틱

In [30]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


## 가. 기준 : 45일


## 1. 모델 생성

In [31]:
X = df_45.drop('label', axis=1)
y = df_45.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [32]:
model = LogisticRegression()   # 로지스틱 회귀 모델 
model.fit(X_train, y_train)    # 모델 학습시키기

y_pred = model.predict(X_test)  # 예측 값

model.score(X_test, y_test)  # 모델 점수

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6826347305389222

In [33]:
print("학습데이터:",model.score(X_train,y_train))
print("테스트데이터:",model.score(X_test,y_test))

학습데이터: 0.6899262111004171
테스트데이터: 0.6826347305389222


In [34]:
model.score(X_test, y_test)

0.6826347305389222

## 2. 평가지표

In [35]:
confusion_matrix(y_test, y_pred)

array([[1118,  265],
       [ 583,  706]], dtype=int64)

In [36]:
ac_score = accuracy_score(y_test, y_pred)
print(ac_score)

pre_score = precision_score(y_test, y_pred, pos_label=1)
print("precision_1:", pre_score)


pre_score = precision_score(y_test, y_pred, pos_label=0)
print("precision_0:", pre_score)


re_score=recall_score(y_test, y_pred, pos_label=1)
print("recall_1 : ",re_score)


re_score=recall_score(y_test, y_pred, pos_label=0)
re_score
print("recall_0 : ",re_score)

f1= f1_score(y_test, y_pred, pos_label=1)
print("F1 Score: ",f1)


f1= f1_score(y_test, y_pred, pos_label=0)
print("F1 Score: ",f1)

0.6826347305389222
precision_1: 0.7270854788877446
precision_0: 0.6572604350382129
recall_1 :  0.547711404189294
recall_0 :  0.8083875632682574
F1 Score:  0.624778761061947
F1 Score:  0.7250324254215305


## 나. 기준 : 50일


## 1. 모델 생성

In [37]:
X = df_50.drop('label', axis=1)
y = df_50.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [38]:
model = LogisticRegression()   # 로지스틱 회귀 모델 
model.fit(X_train, y_train)    # 모델 학습시키기

y_pred = model.predict(X_test)  # 예측 값

model.score(X_test, y_test)  # 모델 점수

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6642964071856288

In [39]:
print("학습데이터:",model.score(X_train,y_train))
print("테스트데이터:",model.score(X_test,y_test))

학습데이터: 0.6804619826756496
테스트데이터: 0.6642964071856288


In [40]:
model.score(X_test, y_test)

0.6642964071856288

## 2. 평가지표

In [41]:
confusion_matrix(y_test, y_pred)

array([[940, 279],
       [618, 835]], dtype=int64)

In [42]:
ac_score = accuracy_score(y_test, y_pred)
print(ac_score)

pre_score = precision_score(y_test, y_pred, pos_label=1)
print("precision_1:", pre_score)


pre_score = precision_score(y_test, y_pred, pos_label=0)
print("precision_0:", pre_score)


re_score=recall_score(y_test, y_pred, pos_label=1)
print("recall_1 : ",re_score)


re_score=recall_score(y_test, y_pred, pos_label=0)
re_score
print("recall_0 : ",re_score)

f1= f1_score(y_test, y_pred, pos_label=1)
print("F1 Score: ",f1)


f1= f1_score(y_test, y_pred, pos_label=0)
print("F1 Score: ",f1)

0.6642964071856288
precision_1: 0.7495511669658886
precision_0: 0.6033376123234917
recall_1 :  0.5746730901582932
recall_0 :  0.771123872026251
F1 Score:  0.6505648617062719
F1 Score:  0.6769895570759813
